In [5]:
#2022.07.14: Get data of Hajime
from scapy.all import *
from datetime import datetime
import zstandard as zstd
import dpkt
import socket
import sys
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

PCAP_FILE = ['/data/darknet/2016/2016-09-30.pcap.zst']
TH_SYN = 0x02
TH_ACK = 0x10
OUTFILE = 'hajime(2).csv'

In [6]:
#Function1: Extract pcap file from .zst compress and read with dpkt --> all packet = packets
def extract_zst(zstfile):
	with open(zstfile, "rb") as f:
		data = f.read()

	dctx = zstd.ZstdDecompressor()
	decompressed = dctx.stream_reader(data)
	packets = dpkt.pcap.Reader(decompressed)
	print("------Extract .zst and read as Pcap Successfull------")
	return packets

In [7]:
#Function2: Get packet from packets (srcip, dstip, ipid, ipttl) (tsport, tdport, tseq, twin)
metaif = ["IPID", "TTL","SRCIP", "DSTIP", "SPORT", "DPORT", "TSEQ", "WINDS"]
def run(packets):
	datalist = []
	#counter = 0
	for ts, buf in packets:
		try:
			eth = dpkt.ethernet.Ethernet(buf)
		except:
			print("Faile parse Frame")

		if type(eth.data) == dpkt.ip.IP:
			ip = eth.data
			srcip 	= int.from_bytes(ip.src, "big")
			dstip	= int.from_bytes(ip.dst, "big")
			ipid 	= ip.id
			ipttl 	= ip.ttl
			if dstip == 2523121634:
				continue
			#Verify: TCP packet
			if ip.p == dpkt.ip.IP_PROTO_TCP:
				tcp 	= ip.data
				try:
					tsport 	= tcp.sport
					tdport 	= tcp.dport
					tseq 	= tcp.seq
					twin 	= tcp.win
					#SYN Packets Only
					if tcp.flags == TH_SYN and tcp.flags != TH_ACK:
						#if tcp.dport == PORT:
						#Hajimeの特徴：　TCP.SEQ番号の上位か下位が０となっている。
						ret1 = tseq & 0x00FF
						ret2 = tseq & 0xFF00
						if twin == 14600:
							if (ret1==0) or (ret2==0):
							#Mirai
							#if dstip == tseq:
								phrase = [ipid, ipttl, srcip, dstip, tsport, tdport, tseq, twin]
								#print(phrase)
								datalist.append(phrase)
				except:
					continue
    
	with open(OUTFILE, "w") as ofile:
		write = csv.writer(ofile)
		write.writerow(metaif)
		write.writerows(datalist)
		print("Packets:\t", end='')
		print(len(datalist))
	ret = pd.DataFrame(data=datalist, columns=metaif)
	return ret

In [8]:
#Function 3: Main
def main(files):
	for i in files:
		print(i)
		plotdata = run(extract_zst(i))

In [9]:
if __name__ == "__main__":
	print("Type path of file")
	main(PCAP_FILE)
	print("Finish")

Type path of file
/data/darknet/2016/2016-09-30.pcap.zst
------Extract .zst and read as Pcap Successfull------
Packets:	113179
Finish
